In [1]:
# 準備工作

from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
import datetime
import re
import sqlite3
import os
import pprint

db_name = "../data/MRT_Taipei.sqlite"
db_url_list = "../data/MRT_Taipei_URLs.sqlite"

In [2]:
# 建立紀錄已經看過的內頁url的資料庫
# 如果這個資料庫已經被建立過了 就略過

# url: 內網url中的ID 具primary key性質
# modDate: 檢視內頁的日期跟時間

if os.path.isfile(db_url_list) == False:

    with sqlite3.connect(db_url_list) as conn_id_list:
        
        c = conn_id_list.cursor()
        
        c.execute("""CREATE TABLE urlList(
        url text unique not null,
        modDate datetime 
        )""")

    conn_id_list.close()

In [3]:
# 建立儲存內頁url跟內頁內容用的資料庫
# 如果這個資料庫已經被建立過了 就略過

# url: 內網url 具primary key性質
# soup: 內頁內容
# nosql: 用於紀錄資料紀錄於nosql資料庫的狀態
# rdb: 用於紀錄資料紀錄於sql資料庫的狀態
# getTime: 紀錄內頁的日期跟時間

if os.path.isfile(db_name) == False:

    with sqlite3.connect(db_name) as conn:
        
        c = conn.cursor()
        
        c.execute("""CREATE TABLE mrtList(
        url text unique not null,
        soup text not null,
        nosql text,
        rdb text,
        getTime datetime 
        )""")
        
    conn.close()

In [4]:
# 定義 remove_dust() 方法
# 移除多餘的字串

def remove_dust(string):
    string = re.sub(" |\n|\r|\xa0","",string)
    return string

In [5]:
# 取外網資料

# 台北捷運
# 不包括桃園捷運

main_url = "http://www.metro.taipei/ct.asp?xItem=78479152&CtNode=70089&mp=122035"
page_source = requests.get(main_url)
soup = BeautifulSoup(page_source.text,'html.parser')

soup_stns = soup.find_all('option')
list_stn_names = [re.split(" ",each.text)[1] for each in soup_stns]
list_stn_IDs = [re.split(" ",each.text)[0] for each in soup_stns]
list_stn_web_IDs = [each['value'] for each in soup_stns]


In [6]:
# 資料集合

dict_stns = tuple(zip(list_stn_IDs,list_stn_web_IDs,list_stn_names))

# 個別車站資料

In [7]:
# 內網

stn_web_ID = "R10-051"
stn_url = "http://web.metro.taipei/c/stationdetail2010.asp?ID=" + stn_web_ID
stn_page_source = requests.get(stn_url)
stn_page_source.encoding = 'utf-8'
stn_soup = BeautifulSoup(stn_page_source.text,"html.parser")

In [8]:
# 定義方法 get_basic_list()
# 取得個別車站的基本資訊

# '車站名稱', '車站地址', '無障礙電梯位置', '詢問處位置', 
# '飲水臺位置', '廁所位置', '假日自行車進出'

def get_basic_list(stn_soup):

    # 車站基本資訊欄位

    basic_attrs = stn_soup.select('th[class="Default"] > img')
    basic_attrs = tuple(remove_dust(each['alt']) for each in basic_attrs)
    basic_attrs_len = len(basic_attrs)


    # 車站基本資訊值

    basic_values = stn_soup.select('td[class="Default"]')
    basic_values = [remove_dust(each.text) for each in basic_values]
    basic_values = basic_values[:basic_attrs_len]


    # 為無障礙電梯位置的資料換行
    # 然後整理成 dictionary
    # (但是有些複雜 先不做太細)
    
#     try:

#         elev_dict = {}
#         elev_index = basic_attrs.index('無障礙電梯位置')
#         split_words = re.findall(">.+?[）]|\d\..+?電梯.+?）|\d\..+?電梯.+?$",basic_values[elev_index])
# #         print(split_words)
#         split_words_count = len(split_words)
        
#         for each in split_words:
#             basic_values[elev_index] = basic_values[elev_index].replace(each, each + '\n')
            
#         basic_values[elev_index] = basic_values[elev_index].split('\n')[:split_words_count]
#         print(basic_values[elev_index])
        
#         for each in basic_values[elev_index]:
#             each_split = each.split('：')
#             elev_dict[each_split[0]] = each_split[1]

#     except Exception as e:
#         print("沒有'無障礙電梯位置'欄位: " + str(e))
    
#     finally:
#         basic_values[elev_index] = elev_dict

    
    # 整合資料    

    basic_values = tuple(basic_values)
    basic_list = [basic_attrs, basic_values]
    
    return basic_list

In [9]:
# 定義 get_exit_list() 方法
# 傳回含有出口資訊的list


def get_exit_list(stn_soup):

    # 找出停車場表格的第一行標題

    desired = stn_soup.select_one('img[alt="出口資訊"]')
    titles = desired.find_parent('p').find_all('th')
    titles = tuple(title.text for title in titles[1::])
    titles_len = len(titles)


    # 用正規表示法找出表格中所有只表示出口編號的tag

    exit_rows_center = stn_soup.find_all('font',text=re.compile('^M\d+?|^出口[\d]+?$'))
#     exit_rows_center = [remove_dust(each.text) for each in exit_rows_center]
#     print(exit_rows_center)

    # 找出同一排中所有的tag

    exit_list = [titles]

    for exit_row_center_tag in exit_rows_center:

        # 找出出口編號於每行的相對位置
        # 然後找出同一行的parent tag

        exit_row_left_pos = titles.index('出口編號')
        exit_row_right_pos = (titles_len) - (titles.index('出口編號') + 1)
#         print(exit_row_center_tag)
        exit_row = exit_row_center_tag.find_parent()

        # 擷取左邊的內容

        exit_row_prev_sibs = exit_row.find_previous_siblings()
        exit_row_prev_sibs = exit_row_prev_sibs[:exit_row_left_pos]


        # 如果左邊有欄位 但是是以圖片表示的話
        # <td> 不會有string 但會有<img>
        # 取img裡的title並存放在一個子list裡
        # 再將子list存到exit_row_prev_sibs中

        if len(exit_row_prev_sibs) == 0:

            exit_row_prev_sibs = ['']

        else:

            # 在同一行一個一個看左邊的表格

            for each_sib in exit_row_prev_sibs:
                sib_index = exit_row_prev_sibs.index(each_sib)
                sib_img_set = each_sib.select('img')

                # 如果該表格中有影像檔
                # 就取出該表格中所有的<img>中的title
                # 並儲存結果到 exit_row_prev_sibs 中

                if len(sib_img_set) > 0:
                    each_img_titles = []
                    each_img_titles = [each_img['title'] for each_img in sib_img_set]
                    exit_row_prev_sibs[sib_index] = each_img_titles
                else:
                    exit_row_prev_sibs[sib_index] = exit_row_prev_sibs[sib_index].text


        # 擷取右邊的內容
        # 原則跟左邊一樣

        exit_row_next_sibs = exit_row.find_next_siblings() 
        exit_row_next_sibs = exit_row_next_sibs[:exit_row_right_pos]

        # 如果右邊有欄位 但是是以圖片表示的話
        # <td> 不會有string 但會有<img>
        # 取img裡的title並存放在一個子list裡
        # 再將子list存到exit_row_next_sibs中

        if len(exit_row_next_sibs) == 0:

            exit_row_next_sibs = ['']

        else:

            # 在同一行一個一個看左邊的表格

            for each_sib in exit_row_next_sibs:
                sib_index = exit_row_next_sibs.index(each_sib)
                sib_img_set = each_sib.select('img')

                # 如果該表格中有影像檔
                # 就取出該表格中所有的<img>中的title
                # 並儲存結果到 exit_row_prev_sibs 中

                if len(sib_img_set) > 0:
                    each_img_titles = []
                    each_img_titles = [each_img['title'] for each_img in sib_img_set]
                    exit_row_next_sibs[sib_index] = each_img_titles
                else:
                    exit_row_next_sibs[sib_index] = exit_row_next_sibs[sib_index].text


        # 出口本身

        exit_row_center = [exit_row_center_tag.text]

        # 整合

        exit_row = tuple(exit_row_prev_sibs + exit_row_center + exit_row_next_sibs)

        # 存到 exit_list 中

        exit_list.append(exit_row)

    return exit_list

In [10]:
# 轉乘停車場資訊
# parking_values
# *** Need to split the rows

def get_parking_list(stn_soup):

    parking = stn_soup.select_one('img[alt="轉乘停車資訊"]')
    parking = parking.find_parent('table')
    parking = parking.find('table')
    parking = parking.find_all('tr')

    parking_list = []

    parking_attrs = tuple(remove_dust(each.text) for each in parking[0].select('th'))
    parking_list.append(parking_attrs)

    for each in parking[1::]:
        parking_values = tuple(remove_dust(every.text) for every in each.select('td'))
        parking_list.append(parking_values)
    
    return parking_list

In [11]:
# 建立 get_stn_dict() 方法
# 整合 基本資訊 出口資訊 轉乘停車資訊
# 並將資料存到stn_dict中並傳回

# 內網

stn_web_ID = "O05-041"
stn_url = "http://web.metro.taipei/c/stationdetail2010.asp?ID=" + stn_web_ID
stn_page_source = requests.get(stn_url)
stn_page_source.encoding = 'utf-8'
stn_soup = BeautifulSoup(stn_page_source.text,"html.parser")

def get_stn_dict(stn_soup):

    stn_dict = {
        '基本資訊': get_basic_list(stn_soup),
        '出口資訊': get_exit_list(stn_soup),
        '轉乘停車資訊': get_parking_list(stn_soup)
    }

    return stn_dict

In [12]:
# 跑迴圈
# 將所有的車站資訊抓下來

all_stn_dict = {}

for each in dict_stns:

    stn_web_ID = each[1]
    stn_name = each[2]
    
    print(stn_web_ID)
    print(stn_name)
    
    stn_url = "http://web.metro.taipei/c/stationdetail2010.asp?ID=" + stn_web_ID
    stn_page_source = requests.get(stn_url)
    stn_page_source.encoding = 'utf-8'
    stn_soup = BeautifulSoup(stn_page_source.text,"html.parser")
    
    stn_dict = get_stn_dict(stn_soup)
    
    all_stn_dict[stn_name] = stn_dict

BR01-019
動物園
BR02-018
木柵
BR03-017
萬芳社區
BR04-016
萬芳醫院
BR05-015
辛亥
BR06-014
麟光
BR07-013
六張犁
BR08-012
科技大樓
BR09-011
大安
BR10-010
忠孝復興
BR11-009
南京復興
BR12-008
中山國中
BR13-007
松山機場
BR14-021
大直
BR15-022
劍南路
BR16-023
西湖
BR17-024
港墘
BR18-025
文德
BR19-026
內湖
BR20-027
大湖公園
BR21-028
葫洲
BR22-029
東湖
BR23-030
南港軟體園區
BR24-031
南港展覽館
R02-099
象山
R03-100
台北101/世貿
R04-101
信義安和
R05-011
大安
R06-103
大安森林公園
R07-134
東門
R08-042
中正紀念堂
R09-050
台大醫院
R10-051
台北車站
R11-053
中山
R12-054
雙連
R13-055
民權西路
R14-056
圓山
R15-057
劍潭
R16-058
士林
R17-059
芝山
R18-060
明德
R19-061
石牌
R20-062
唭哩岸
R21-063
奇岩
R22-064
北投
R22A-065
新北投
R23-066
復興崗
R24-067
忠義
R25-068
關渡
R26-069
竹圍
R27-070
紅樹林
R28-071
淡水
G01-033
新店
G02-034
新店區公所
G03-035
七張
G03A-032
小碧潭
G04-036
大坪林
G05-037
景美
G06-038
萬隆
G07-039
公館
G08-040
台電大樓
G09-041
古亭
G10-042
中正紀念堂
G11-043
小南門
G12-086
西門
G13-105
北門
G14-053
中山
G15-132
松江南京
G16-009
南京復興
G17-109
台北小巨蛋
G18-110
南京三民
G19-111
松山
O01-048
南勢角
O02-047
景安
O03-046
永安市場
O04-045
頂溪
O05-041
古亭
O06-134
東門
O07-089
忠孝新生
O08-132
松江南京
O09-131
行天宮
O10-

In [13]:
pprint.pprint((all_stn_dict))

{'七張': {'出口資訊': [('無障礙出口', '出口編號', '位置描述', '出入口型式', 'Google地圖', '備註'),
                 (['無障礙出口'],
                  '出口1',
                  '北新路2段150號',
                  ['樓梯'],
                  ['查看Google地圖(另開新視窗)'],
                  ''),
                 ('',
                  '出口2',
                  '北新路2段、鄰近明德路口、近新店郵局',
                  ['樓梯'],
                  ['查看Google地圖(另開新視窗)'],
                  '')],
        '基本資訊': [('車站名稱',
                  '車站地址',
                  '無障礙電梯位置',
                  '詢問處位置',
                  '飲水臺位置',
                  '廁所位置',
                  '假日自行車進出'),
                 ('七張',
                  '23143新北市新店區北新路2段150號',
                  '月臺電梯：出口1無障礙閘門旁、第2月臺中央',
                  '近出口1',
                  '出口1',
                  '付費區，大廳層電扶梯旁',
                  '開放')],
        '轉乘停車資訊': [('小汽車', '機車', '自行車'), ('0', '0', '0')]},
 '三和國中': {'出口資訊': [('無障礙出口', '出口編號', '位置描述', '出入口型式', 'Google地圖', '備註'),
                   (['無障礙出口'],
  

                  ('劍南路',
                   '10464臺北市中山區北安路798號',
                   '1.出口電梯：出口1臨北安路、出口2臨北安路2.月臺電梯：月臺層東側',
                   '近出口2、3',
                   '近出口2、3',
                   '付費區，近出口2、3',
                   '未開放')],
         '轉乘停車資訊': [('小汽車', '機車', '自行車', 'Google地圖'),
                    ('282', '52', '122', '')]},
 '劍潭': {'出口資訊': [('無障礙出口',
                  '出口編號',
                  '位置描述',
                  '出入口型式',
                  'Google地圖',
                  '環景照片',
                  'Youbike',
                  '備註'),
                 ('',
                  '出口1',
                  '中山北路5段65號',
                  ['樓梯'],
                  ['查看Google地圖(另開新視窗)'],
                  ['查看出口1環景照片(另開新視窗)'],
                  '',
                  ''),
                 (['無障礙出口'],
                  '出口2',
                  '基河路22號對面',
                  ['樓梯'],
                  ['查看Google地圖(另開新視窗)'],
                  ['查看出口2環景照片(另開新視窗)'],
                  

                    ''),
                   ('', 'M6', '忠孝西路1段38號', ['樓梯'], ['查看Google地圖(另開新視窗)'], ''),
                   ('', 'M7', '忠孝西路1段33號', ['樓梯'], ['查看Google地圖(另開新視窗)'], ''),
                   ('',
                    'M8',
                    '公園路13號',
                    ['向上及向下電扶梯'],
                    ['查看Google地圖(另開新視窗)'],
                    '')],
          '基本資訊': [('車站名稱',
                    '車站地址',
                    '無障礙電梯位置',
                    '詢問處位置',
                    '飲水臺位置',
                    '廁所位置',
                    '假日自行車進出'),
                   ('台北車站',
                    '10041臺北市中正區忠孝西路1段49號',
                    '1號電梯：B2層（板南線大廳中央偏北處）>B4層（淡水信義線2號月臺南側）2號電梯：B3層（淡水信義線大廳中央）>B4層（淡水信義線月臺中央）3號電梯：地面層（出口M1及出口M2中間）>B3層（淡水信義線大廳北處）4號電梯：B1層（出口M4及出口M5中間）>B2層（板南線大廳西側）5號電梯：出口M4（忠孝西路靠臺鐵側）>B1層（星巴克）6號電梯：B2層（板南線大廳中央處）>B3層（板南線月臺中央）7號電梯：B1層（誠品書店）>B2層（板南線大廳東側）',
                    '近出口M3、M7、M8，近忠孝西路近出口M4、M5、M6，近忠孝西路近出口M1、M2，近市民大道',
                    '出口M4',
      

                  '出口1',
                  '文德路(內湖高中對面)',
                  ['電梯', '樓梯', '電扶梯向上'],
                  ['查看Google地圖(另開新視窗)'],
                  ''),
                 (['無障礙出口'],
                  '出口2',
                  '文德路(內湖高中旁)',
                  ['電梯', '樓梯', '向上及向下電扶梯'],
                  ['查看Google地圖(另開新視窗)'],
                  '')],
        '基本資訊': [('車站名稱',
                  '車站地址',
                  '無障礙電梯位置',
                  '詢問處位置',
                  '飲水臺位置',
                  '廁所位置',
                  '假日自行車進出'),
                 ('文德',
                  '11475臺北市內湖區文德路214號',
                  '1.出口電梯：出口1（內湖高中對面）及出口2（內湖高中旁）2.月臺電梯：月臺層東側',
                  '近出口2',
                  '出口2',
                  '非付費區，近出口2',
                  '未開放')],
        '轉乘停車資訊': [('小汽車', '機車', '自行車', 'Google地圖'), ('306', '0', '36', '')]},
 '新北投': {'出口資訊': [('無障礙出口',
                   '出口編號',
                   '位置描述',
                   '出入口型式',
                   'Google

                  '假日自行車進出'),
                 ('石牌',
                  '11271臺北市北投區石牌路1段200號',
                  '月臺電梯：站體中央處、站體南側',
                  '近出口1近出口2',
                  '出口1',
                  '付費區，近出口1付費區，近出口2',
                  '開放')],
        '轉乘停車資訊': [('小汽車', '機車', '自行車', 'Google地圖'), ('64', '256', '582', '')]},
 '科技大樓': {'出口資訊': [('無障礙出口',
                    '出口編號',
                    '位置描述',
                    '出入口型式',
                    'Google地圖',
                    '備註',
                    '小汽車',
                    '機車',
                    '自行車',
                    'Google地圖')],
          '基本資訊': [('車站名稱',
                    '車站地址',
                    '無障礙電梯位置',
                    '詢問處位置',
                    '飲水臺位置',
                    '廁所位置',
                    '假日自行車進出'),
                   ('科技大樓',
                    '10667臺北市大安區復興南路2段235號',
                    '1.出口電梯：單一出口2.月臺電梯：月臺層南側',
                    '近出口，3樓大廳層',
                    '出口

In [443]:
# 抓單一車站至所有車站的時間

time_table_url = "http://web.metro.taipei/c/TicketALLresult.asp"
time_table_source = requests.get(time_table_url)
time_table_source.encoding = 'utf-8'
time_table_soup = BeautifulSoup(time_table_source.text,"html.parser")


In [444]:
stn_optgroup = stn_soup.find_all('option',value=True)
stn_optgroup = [each.text for each in stn_optgroup]
# (stn_optgroup)

In [445]:
time_table_top_col = tuple(each.text for each in time_table_soup.select('th'))

In [19]:
url = "http://web.metro.taipei/c/TicketALLresult.asp"
payload = {"s2elect" : "BR01-019", "submit" : "確定"}
head = {"User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}
res = requests.post(url, headers = head, data = payload)
res.encoding = 'utf8'

In [20]:
soup = BeautifulSoup(res.text, 'html.parser')
soup


<!DOCTYPE html>

<html lang="tw" x-frame-options="sameorigin">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=EmulateIE7" http-equiv="X-UA-Compatible"/>
<title>臺北大眾捷運股份有限公司 ─ 票價及乘車時間</title>
<meta content="票價及乘車時間" name="DC.Title"/>
<meta content="票價及乘車時間" name="DC.Subject"/>
<meta content="捷運公司網站管理員" name="DC.Creator"/>
<meta content="若您無法讀取PDF格式文件，請先下載 Acrobat Reader 安裝程式 (另開視窗)" name="DC.Description"/>
<meta content="臺北大眾捷運股份有限公司" name="DC.Publisher"/>
<meta content="臺北大眾捷運股份有限公司" name="DC.Contributor"/>
<meta content="2012-09-05" name="DC.coverage.t.max"/>
<meta content="2009-09-05" name="DC.coverage.t.min"/>
<meta content="2009-10-28 00:00" name="DC.Date"/>
<meta content="HTML" name="DC.Type"/>
<meta content="文字" name="DC.Format"/>
<meta content="379620000M" name="DC.Identifier"/>
<meta content="CP-1017531" name="DC.Relation"/>
<meta content="臺北大眾捷運股份有限公司" name="DC.Source"/>
<meta content="中文" name="DC.Language"/>
<meta content="臺北大眾

In [448]:
time_table_soup


<!DOCTYPE html>

<html lang="tw" x-frame-options="sameorigin">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=EmulateIE7" http-equiv="X-UA-Compatible"/>
<title>臺北大眾捷運股份有限公司 ─ 票價及乘車時間</title>
<meta content="票價及乘車時間" name="DC.Title"/>
<meta content="票價及乘車時間" name="DC.Subject"/>
<meta content="捷運公司網站管理員" name="DC.Creator"/>
<meta content="若您無法讀取PDF格式文件，請先下載 Acrobat Reader 安裝程式 (另開視窗)" name="DC.Description"/>
<meta content="臺北大眾捷運股份有限公司" name="DC.Publisher"/>
<meta content="臺北大眾捷運股份有限公司" name="DC.Contributor"/>
<meta content="2012-09-05" name="DC.coverage.t.max"/>
<meta content="2009-09-05" name="DC.coverage.t.min"/>
<meta content="2009-10-28 00:00" name="DC.Date"/>
<meta content="HTML" name="DC.Type"/>
<meta content="文字" name="DC.Format"/>
<meta content="379620000M" name="DC.Identifier"/>
<meta content="CP-1017531" name="DC.Relation"/>
<meta content="臺北大眾捷運股份有限公司" name="DC.Source"/>
<meta content="中文" name="DC.Language"/>
<meta content="臺北大眾

In [446]:
# driver = webdriver.PhantomJS()
# driver.get(time_table_url)
# time.sleep(2.5)
# page_source = str(driver.page_source)

In [ ]:
# page_source

In [350]:
# 定義 insert_urls() 方法

# 將爬下來的ID存到爬過的url紀錄中

# 參數: 
# db_name: 要塞的資料庫名稱(sqlite檔案)
# id_list: 含有內頁ID的list

def insert_urls(db_url_list, id_list):
    
    with sqlite3.connect(db_url_list) as conn_insert_url:
               
        c_iu = conn_insert_url.cursor()

        print(id_list)
        
        for each_id in id_list:
        
            try:
        
                today = datetime.datetime.today()
                today = today.strftime('%Y-%m-%d %H:%M:%S')

                values = "'http://www.century21.com.tw/index/Rental/RentPage/" + str(each_id) + "','" + str(today) + "'"       

                insertString = "INSERT INTO urlList (url, modDate) VALUES(" + values + ");"
                c_iu.execute(insertString)
            
            except Exception as e:
                
                print("例外: " + str(e))
                print(each_id)
            
    c_iu.close()

In [ ]:
# 找出所有已經被爬過的內頁url

with sqlite3.connect(db_name) as retr_url:
        
    c = retr_url.cursor()

    c.execute("SELECT url from rental")
    url_completed = c.fetchall()
    
retr_url.close()
# url_completed[0][0][-6:]

In [ ]:
# 定義 crawl_links_single() 方法
# 從一個外網頁面找出該頁面裡所有的內頁ID

def crawl_links_single(driver):

    soup = BeautifulSoup(driver.page_source,'html.parser')
    soup_hrefs = soup.select('div[class="main clearfix"] > div > a')
    hrefs = []

    for each in soup_hrefs:
        hrefs.append(each['href'].split('/')[-1])
#         hrefs.append(each['href'])

    return hrefs

In [ ]:
# 定義 crawl_n_insert() 方法
# 爬內頁然後儲存內容到資料庫中

def crawl_n_insert(url, db_name):
    
#     driver = webdriver.PhantomJS()
#     driver.get(url)
#     time.sleep(2.5)
#     page_source = str(driver.page_source)
    page_source = requests.get(url)
    page_source = str(page_source.text)
    
    # 處理引號
    
    page_source = page_source.replace("\"","\"\"")
    page_source = page_source.replace('\'',"\'\'")
    
    with sqlite3.connect(db_name) as conn_insert_page:
               
        c_ip = conn_insert_page.cursor()
        
        try:

            today = datetime.datetime.today()
            today = today.strftime('%Y-%m-%d %H:%M:%S')

            values = "\"" + str(url) + "\",'" + page_source + "','','','" + str(today) + "'"

#             print(values)

            insertString = "INSERT INTO rental (url, soup, nosql, rdb, getTime) VALUES(" + values + ");"

#             print(insertString)

            c_ip.execute(insertString)

        except Exception as e:

            print(url)
            print("例外: " + str(e))
        finally:
            c_ip.close()

In [ ]:
# 主程序1:
# 從外網把所有的內頁url爬下來

main_url = "http://www.century21.com.tw/index/Rental/Rent"

end_loop = False
driver = webdriver.PhantomJS()
driver.get(main_url)
time.sleep(2.5)

while not end_loop:
    
    try:
    
        print(driver.current_url)
        id_list = crawl_links_single(driver)
        return_count = len(id_list)

        if return_count > 0:
            insert_urls(db_url_list, id_list)
    
        driver.find_element_by_link_text("下一頁").click()
        time.sleep(1)
        
    except Exception as e:
        
        print("例外: " + str(driver.current_url) + " \r; " + str(e))
        end_loop = True
        
print("內頁url搜尋結束")

In [ ]:
# 檢查 db_url_list 結果

with sqlite3.connect(db_url_list) as conn_url_list:
    
    c_ul = conn_url_list.cursor()
    qryString = "SELECT distinct count(*) from urlList limit 1;"
    c_ul.execute(qryString)
    results = c_ul.fetchall()

c_ul.close()

print((results))

In [ ]:
# 主程序2:
# 從外網把所有的內頁內容爬下來

# 取得所有的url

with sqlite3.connect(db_url_list) as conn_call_list:
    
    c_cl = conn_url_list.cursor()
    qryString = "SELECT url from urlList;"
    c_cl.execute(qryString)
    results = c_cl.fetchall()

c_cl.close()

# 略過已經做過的網址
        
for each in results:
    try:
        print(each[0])
        crawl_n_insert(each[0],db_name)
    except Exception as e:
        print(e)

print("所有的內頁內容都已爬過且儲存到資料庫中")

In [ ]:
# 檢查 db_name 結果

with sqlite3.connect(db_name) as conn_db_name:
    
    c_dn = conn_db_name.cursor()
    qryString = "SELECT count(url) from rental;"
    c_dn.execute(qryString)
    results = c_dn.fetchall()

c_dn.close()
    
print((results))